#### Use iris dataset. Do the pre-processing of the dataset if required. Create Multi-layer Perceptron to predict the flower category. Check for the overfitting of your model.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [2]:
data = pd.read_csv('https://raw.githubusercontent.com/rahul96rajan/sample_datasets/master/iris.csv')
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
X = data.drop('species', axis=1)
y = data['species']

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [5]:
stan_scale = StandardScaler()
stan_scale.fit(X_train)

StandardScaler()

In [6]:
X_train = stan_scale.transform(X_train)
X_test = stan_scale.transform(X_test)

In [7]:
mlp_clf = MLPClassifier(random_state=42, max_iter=500)

In [8]:
mlp_clf.fit(X_train, y_train)

MLPClassifier(max_iter=500, random_state=42)

In [9]:
y_train_pred = mlp_clf.predict(X_train)
print("Train :: {0} \nCV :: {1}".format(f1_score(y_train, y_train_pred,
                                                 average='weighted'), 
                                       np.mean(cross_val_score(mlp_clf, X_train, y_train, cv=10,
                                                              scoring='f1_weighted', n_jobs=-1))))

Train :: 0.9833333333333333 
CV :: 0.923169793169793


In [10]:
mlp = MLPClassifier(random_state=42, max_iter=2000)

params = dict(alpha=[0.1, 0.5, 0.8],
             hidden_layer_sizes=[(6,3), (5,4)])

gscv = GridSearchCV(mlp, params, scoring='accuracy', n_jobs=-1, cv=20)

In [11]:
gscv.fit(X_train, y_train)

GridSearchCV(cv=20, estimator=MLPClassifier(max_iter=2000, random_state=42),
             n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 0.8],
                         'hidden_layer_sizes': [(6, 3), (5, 4)]},
             scoring='accuracy')

In [12]:
print('Score :: {1}\nEstimator :: {0}'.format(gscv.best_estimator_ ,
                                              gscv.best_score_))

Score :: 0.9583333333333334
Estimator :: MLPClassifier(alpha=0.1, hidden_layer_sizes=(5, 4), max_iter=2000,
              random_state=42)


In [13]:
y_pred_train = gscv.predict(X_train)
y_pred_test = gscv.predict(X_test)

values = [[y_train, y_pred_train], [y_test, y_pred_test]]

In [14]:
accu_scores = map(lambda x: round(accuracy_score(x[0], x[1])*100, 2), values)
f1_mic_scores = map(lambda x: round(f1_score(x[0], x[1], average='micro')*100, 2),
                    values)

In [15]:
metric_df = pd.DataFrame({"Accuracy": accu_scores,
                          "F1 Score (micro)": f1_mic_scores},
                         index=['Training', 'Testing'])
display(metric_df)

,Accuracy,F1 Score (micro)
Training,97.5,97.5
Testing,100.0,100.0
